## Ddip User Guide 

### The Two Approaches to making objects available in local and remote processes by the same names:

**1. Create in one namespace, then `%dipush` to, or `%dipull` from the other.**

**2. Create the object simultaneously in both local and DDP processes, with `%%dip everwhere` cell magic**, i.e. *code once, runs everywhere*

NOTE: Since `Ddip` uses `ipyparallel` as the underlying fabric of multiple processes, `ipyparallel`'s family of line and cell magics for parallel computing are also accessible, when `Ddip` extension is loaded.  `Ddip`'s purpose is to provide a narrow interface good enough to run Distributed Data Parallel training applications.  For those who wish to know more about `ipyparallel`, here is [The `ipyparallel` official documentation](https://ipyparallel.readthedocs.io/en/latest/intro.html).

### From local to DDP:

#### By default a cell runs in local notebook namespace, unless it begins with the cell magic `%%dip`. 
Any objects created here, are not visible in the remote processes.

In [1]:
%load_ext Ddip
%makedip -r -g all --verbose False

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-ippdpp_c-client.json


In [2]:
%%dip everywhere
import os # For convenience

In [4]:
foobar = { 1 : "foo", 2 : "bar" }
def var_printer(f): print(f"Process [{os.getpid()}] variable is :{f}")

var_printer(foobar)

Process [16359] variable is :{1: 'foo', 2: 'bar'}


**`foobar` and `var_printer` are not visible in the DDP remote processes, unless we `%dipush` them over:**

In [5]:
%%dip
foobar, var_printer

RemoteError: NameError(name 'foobar' is not defined)

In [6]:
%dipush?

In [7]:
%dipush foobar var_printer

In [8]:
%%dip --see all
# Use `--see all` to see the outputs from all DDP processes
var_printer(foobar)

Process [16413] variable is :{1: 'foo', 2: 'bar'}
Process [16417] variable is :{1: 'foo', 2: 'bar'}
Process [16418] variable is :{1: 'foo', 2: 'bar'}


### From remote DDP processes to local:

**Now in reverse, create something remotely.  They are not visible until we `%dipull` them:**


In [20]:
%%dip --see all
remote_pid = os.getpid()
print(f"Process [{os.getpid()}]: remote_pid is {remote_pid}")

Process [16413]: remote_pid is 16413
Process [16417]: remote_pid is 16417
Process [16418]: remote_pid is 16418


In [22]:
remote_pid

NameError: name 'remote_pid' is not defined

**`%dipull` by default pulls in the object from RANK 0 process**

In [23]:
%dipull remote_pid
print(f"Process [{os.getpid()}]: remote_pid is {remote_pid}")

Process [16359]: remote_pid is 16413


In [24]:
%dipull -r 1 remote_pid
print(f"Process [{os.getpid()}]: remote_pid is {remote_pid}")

Process [16359]: remote_pid is 16417


### The second way of making objects available in both remote and local processes by name, is to create copies of them in all processes simultaneously.

**This is done with `%%dip everywhere` cell magic.  It is particularly useful to import libraries in both local and remote processes, in one shot.**

**Obviously the value of object X can be different from process to process, and this trick is mostly a syntatic sugar to disseminate identical piece of code and objects with the same name everywhere.*

In [28]:
%%dip everywhere --see all
x = [ f"Different pid: {os.getpid()}", 2, ["a", "list"], {"and": "a dict"}]
x

['Different pid: 16359', 2, ['a', 'list'], {'and': 'a dict'}]

Out[0:10]: ['Different pid: 16413', 2, ['a', 'list'], {'and': 'a dict'}]

Out[1:10]: ['Different pid: 16417', 2, ['a', 'list'], {'and': 'a dict'}]

Out[2:10]: ['Different pid: 16418', 2, ['a', 'list'], {'and': 'a dict'}]